In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hw4_2 import *
from torch.optim import Adam,SGD
from torch.autograd import Variable
from tqdm.notebook import tqdm
import datetime

In [3]:
batch_size = 128
lr = 3e-4

In [4]:
model = loadModel('BYOL_backbone_500.pt',fixedbackbone=True)

train_loader = loadData('hw4_data/office/train',batch_size,'hw4_data/office/train.csv',num_workers=6,finetune=True)
val_loader = loadData('hw4_data/office/val',512,'hw4_data/office/val.csv',num_workers=6,finetune=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=lr)

In [5]:
def saveModel(name):
    path = name
    torch.save(model.state_dict(), path)

def valAccuracy():
    model.eval().cuda()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = model(images.cuda())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels.cuda()).sum().item()
    
    accuracy = (100 * accuracy / total)
    return(accuracy)


def train(num_epochs):
    model.train()
    best_accuracy = 0.0

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        pbar = tqdm(train_loader)
        for (images, labels) in pbar:
            
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()
            pbar.set_description(f"loss: {loss.item():.4f}")

        # Compute and print the average accuracy fo this epoch when tested over all test images
        accuracy = valAccuracy()
        print('For epoch', epoch+1,': test accuracy:{:.4f}%, loss:{:.4f}'.format(accuracy,loss.item()))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            #saveModel('C_'+str(accuracy)[:5]+'.pth')
            best_accuracy = accuracy
            print("###BEST:",best_accuracy)

In [6]:
train(50)

The model will be running on cuda:0 device


  0%|          | 0/31 [00:00<?, ?it/s]

For epoch 1 : test accuracy:12.5616%, loss:3.5948
###BEST: 12.561576354679802


  0%|          | 0/31 [00:00<?, ?it/s]

For epoch 2 : test accuracy:13.3005%, loss:3.2986
###BEST: 13.300492610837438


...

For epoch 48 : test accuracy:25.6158%, loss:2.3918


  0%|          | 0/31 [00:00<?, ?it/s]

For epoch 49 : test accuracy:26.3547%, loss:2.6205


  0%|          | 0/31 [00:00<?, ?it/s]

For epoch 50 : test accuracy:25.8621%, loss:2.5480
